In [2]:
!pwd

/Users/hugocoussens/git/betfair_profitbox/research


In [3]:
from pathlib import Path
import json, csv
from pathlib import Path


paths = [str(p) for p in Path("./ADVANCED/").rglob("*") if p.is_file()]
paths = [p for p in paths if not p.lower().endswith(".ds_store")]
matches = []

for path in paths:
    with open(path, "r") as f:
        try:
            for line in f:
                if '"marketType":"MATCH_ODDS"' in line and '"status":"ACTIVE"' and '"name":"Match Odds"' in line:
                    matches.append(line.strip())
        except: print(path)


lines = matches  # your list of JSON strings

def safe_load_json(s):
    try: return json.loads(s)
    except: return json.loads(s[s.index("{"):])

unique = {}
for ln in lines:
    o = safe_load_json(ln)
    for mc in o.get("mc", []):
        md = mc.get("marketDefinition", {})
        k = (md.get("eventId"), mc.get("id"))
        if all(k):
            if md["runners"][0]["status"] == 'ACTIVE' and len(md["runners"]) ==2 and md["runners"][0]["name"] not in ("No", "Yes"):
                runners = (md["runners"])
                unique[k] = {"eventName": md.get("eventName"), "id": mc.get("id"), "eventId": md.get("eventId"), "runners": runners}

with open("input.csv","w",newline="",encoding="utf-8") as f:
    csv.DictWriter(f,fieldnames=["eventName","id","eventId","runners"]).writeheader(); f.writelines(
        ",".join(map(str,row.values()))+"\n" for row in unique.values()
    )

 # Gets all MATCH_ODDS event and market ids into csv brute force  ^^^


files = {Path(p).name: p for p in paths}
nlines = lambda p: sum(1 for _ in open(p, "rb"))

with open("input.csv", encoding="utf-8") as f:
    header = f.readline().rstrip("\n")
    h1,h2,h3,_ = header.split(",", 3)  # eventName,id,eventId,runners
    lines = [ln.rstrip("\n") for ln in f]

bad = set()
for ln in lines:
    try:
        _, idv, _, _ = ln.split(",", 3)
        fp = files.get(idv)
        if fp and nlines(fp) < 200:
            bad.add(idv)
    except ValueError:
        continue  # skip malformed rows

with open("ADVeventid_marketid_AUG24.csv", "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for ln in lines:
        try:
            _, idv, _, _ = ln.split(",", 3)
            if idv not in bad:
                f.write(ln + "\n")
        except ValueError:
            pass

 # Cleans up to only have market ids with >200 lines (aka more likely the actual match odds) - wouldnt work on more granual tick data files)

with open("ADVeventid_marketid_AUG24.csv", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    rows = list(reader)
with open("ADVeventid_marketid_AUG24.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    for count, row in enumerate(rows):
        if count >0:
            row.append([x for x in files.values() if row[1] in x][0])
            writer.writerow(row)
        else:
            row.append("path")
            writer.writerow(row)
            

In [4]:
import logging
from dotenv import load_dotenv
import os
import betfairlightweight
from betfairlightweight import StreamListener


# setup logging
logging.basicConfig(level=logging.INFO)

# Access the environment variables
# load .env
load_dotenv(dotenv_path="/Users/hugocoussens/git/betfair_profitbox/.env")

USERNAME = os.getenv("BETFAIR_USERNAME")
APP_KEY  = os.getenv("BETFAIR_APP_KEY")
PASSWORD = os.getenv("BETFAIR_PASSWORD")
print(USERNAME, APP_KEY, PASSWORD)

trading = betfairlightweight.APIClient(username=USERNAME, password=PASSWORD,
                                       app_key=APP_KEY,cert_files=("/Users/hugocoussens/git/betfair_profitbox/certs/client-2048.crt",
                                                                   "/Users/hugocoussens/git/betfair_profitbox/certs/client-2048.key"))

# create listener
listener = StreamListener(max_latency=None)


# print them all
for i, p in id_paths.items():
    print(i, "→", p)

    # create historical stream (update file_path to your file location)
    stream = trading.streaming.create_historical_generator_stream(
        file_path=p,
        listener=listener,)

    # create generator
    gen = stream.get_generator()

    # print marketBooks
    for market_books in gen():
        for market_book in market_books:
            print(dir(market_book))

    # print based on seconds to start
    for market_books in gen():
        for market_book in market_books:
            seconds_to_start = (
                market_book.market_definition.market_time - market_book.publish_time
            ).total_seconds()
            if seconds_to_start < 100:
                print(market_book.market_id, seconds_to_start, market_book.total_matched)
    
            # print winner details once market is closed
            if market_book.status == "CLOSED":
                for runner in market_book.runners:
                    if runner.status == "WINNER":
                        print(
                            "{0}: {1} with sp of {2}".format(
                                runner.status, runner.selection_id, runner.sp.actual_sp
                            )
                        )

    # record prices to a file
    with open(f"./aug_market_books/{i}.csv", "w") as output:
        output.write("Time,MarketId,Status,Inplay,SelectionId,LastPriceTraded\n")
    
    for market_books in gen():
        for market_book in market_books:
            with open(f"./aug_market_books/{i}.csv", "a") as output:
                for runner in market_book.runners:
                    # how to get runner details from the market definition
                    market_def = market_book.market_definition
                    runners_dict = {
                        (runner.selection_id, runner.handicap): runner
                        for runner in market_def.runners
                    }
                    runner_def = runners_dict.get((runner.selection_id, runner.handicap))
    
                    output.write(
                        "%s,%s,%s,%s,%s,%s\n"
                        % (
                            market_book.publish_time,
                            market_book.market_id,
                            market_book.status,
                            market_book.inplay,
                            runner.selection_id,
                            runner.last_price_traded or "",
                        )
                    )

None None None


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'